In [1]:
import tensorflow as tf
import numpy as np
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from random import shuffle
import collections
from collections import namedtuple

[nltk_data] Downloading package punkt to /home/hamza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def fileRead(fname):
    
    with open(fname) as f:
        content = f.readlines()
    return content
fname='/students/u6019739/DocumentAnalysis/Assignment3/starter_code_data/sentences_train.txt'
lname='/students/u6019739/DocumentAnalysis/Assignment3/starter_code_data/labels_train.txt'
devName='/students/u6019739/DocumentAnalysis/Assignment3/starter_code_data/sentences_dev.txt'
devlabel='/students/u6019739/DocumentAnalysis/Assignment3/starter_code_data/labels_dev.txt'
train_ngrams=fileRead(fname);
train_labels=fileRead(lname);   
test_ngrams=fileRead(devName);
test_labels=fileRead(devlabel);

FileNotFoundError: [Errno 2] No such file or directory: '/students/u6019739/DocumentAnalysis/Assignment3/starter_code_data/sentences_train.txt'

In [ ]:
Ngram = namedtuple('Ngram', 'sentence classs')

In [ ]:
#done
def tokenize(ngram):
    return word_tokenize(ngram)

In [ ]:
# this function takes the ngrams and provides two maps key value, value key. key=unique words value=rank according to freq.
#done
def build_vocab(train_set):
    words = list()
    for ngram in train_set:
        tokens = tokenize(ngram)
        words.extend(tokens)    
    count = collections.Counter(words).most_common()
    word_to_id = dict()
    word_to_id['PAD'] = 0
    for word, _ in count:
        word_to_id[word] = len(word_to_id)
    id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))
    return word_to_id, id_to_word
#build_vocab(train_ngrams)
#build_vocab(train_labels)

In [ ]:
#done
def map_token_seq_to_word_id_seq(token_seq, word_to_id):
    return [map_word_to_id(word_to_id,word) for word in token_seq]

def map_word_to_id(word_to_id, word):
    if word in word_to_id:
        return word_to_id[word]
    else:
        return word_to_id['PAD']

In [ ]:
def build_dataset(train_set, word_to_id,label_set,label_to_id):
    dataset = list();
    i=0;
    for ngram in train_set:
        tokens = tokenize(ngram)
        tokens1= tokenize(label_set[i])
        word_id_seq = map_token_seq_to_word_id_seq(tokens, word_to_id) # gives sequence numbers(list) of every token
        label= label_to_id[tokens1[0]]
        i=i+1;
        ngram_inst = Ngram(sentence=word_id_seq, classs=label)
        dataset.append(ngram_inst)
    return dataset
word_to_id, id_to_word = build_vocab(train_ngrams)
label_to_id, id_to_label =build_vocab(train_labels)
train_set=build_dataset(train_ngrams,word_to_id,train_labels,label_to_id)
test_set=build_dataset(test_ngrams,word_to_id,test_labels,label_to_id)

In [ ]:
def convert_to_label_vec(label_id, num_labels):
    # initialise a zero vector of the length num_words
    label_vec = [0] * num_labels
    label_vec[label_id] = 1
    return label_vec

In [ ]:
def train_eval(word_to_id,label_to_id,train_dataset, dev_dataset,test_dataset, num_epochs=3, learning_rate=0.1, embedding_dim=10):
    num_words = len(word_to_id)
    num_labels = len(label_to_id)
    # Placeholders are inputs of the computation graph. 
    input_ngram = tf.placeholder(tf.int32, shape = [None])
    input_sens = tf.placeholder(tf.int32, shape = [None])
    correct_label = tf.placeholder(tf.float32, shape=[num_labels]) 
    # Word embeddings are the only parameters of the model
  
    # bias is not needed for embeddings
    # b = tf.Variable(tf.zeros([num_words]))
    with tf.Session() as sess:
        embeddings = tf.Variable(tf.random_uniform([num_words, embedding_dim], -1.0, 1.0)) # creates a 2 dim array of random numbers with dim(num of words x 10)
        embeddings1 = tf.Variable(tf.random_uniform([embedding_dim,num_labels], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, input_ngram)
        tmp_m = tf.reduce_sum(embed, 0)
        sum_rep = tf.reshape(tmp_m, [1, embedding_dim])
        # Formulate word embedding learning as a word prediction task. Note that, no negative sampling is applied here.
        y = tf.nn.softmax(tf.matmul(sum_rep, embeddings1))
        c=tf.log(y);
        d=-tf.reduce_sum(correct_label * tf.log(y), reduction_indices=[1])
#        cross_entropy = tf.reduce_mean(-tf.reduce_sum(correct_label * tf.log(y), reduction_indices=[1]))
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(correct_label * tf.log(tf.clip_by_value(y,1e-10,1.0)), reduction_indices=[1]))
        
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(correct_label, 0))
        accuracy = tf.cast(correct_prediction, tf.float32)

        sess.run(tf.initialize_all_variables())
        
        # Build SGD optimizer
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
        for epoch in range(num_epochs):
            shuffle(train_dataset)
            for ngram_inst in train_dataset:
                # Run one step of SGD to update word embeddings.
                train_step.run(feed_dict={input_ngram: ngram_inst.sentence, correct_label: convert_to_label_vec(ngram_inst.classs, num_labels)})
      
            print('Epoch %d : %s .' % (epoch,compute_accuracy(num_labels, accuracy,input_ngram, correct_label, dev_dataset)))
        print('Accuracy on test set %d : %s .' %(epoch,compute_accuracy(num_labels, accuracy,input_ngram, correct_label, test_dataset)))
        # input_sens is the placeholder of an input sentence.
#        test_results = predict(prediction, input_sens, test_dataset)
    return embeddings,embeddings1



In [ ]:
def compute_accuracy(num_labels, accuracy,input_ngram, correct_label, eval_dataset):
    num_correct = 0
    for ngram_inst in eval_dataset:
        num_correct += accuracy.eval(feed_dict={input_ngram: ngram_inst.sentence, correct_label: convert_to_label_vec(ngram_inst.classs, num_labels)})
    print('# of correct sentences are %s ' % num_correct)
    return num_correct / len(eval_dataset)

In [ ]:
embeddings1,embeddings2=train_eval(word_to_id,label_to_id,train_set, train_set,test_set,num_epochs=3, learning_rate=0.1, embedding_dim=10)
